In [1]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [19]:
import faiss
import numpy as np

# 初始化数据
d = 128  # 向量维度
nb = 10000  # 数据库大小
nq = 100  # 查询向量数量
np.random.seed(1234)
db_vectors = np.random.random((nb, d)).astype('float32')
query_vectors = np.random.random((nq, d)).astype('float32')

# FAISS资源和索引配置
res = faiss.StandardGpuResources()  # 使用标准GPU资源
index_flat = faiss.IndexFlatIP(d)  # 使用内积作为距离度量

# 创建GpuIndexIVFPQ索引
nlist = 100  # 聚类中心的数量
m = 16  # 子向量数量
k = 200  # 要返回的最近邻数量
quantizer = index_flat  # 使用IndexFlatIP作为量化器
# index = faiss.index_cpu_to_gpu(res, 0, faiss.IndexIVFPQ(quantizer, d, nlist, m, 8))
index = faiss.index_cpu_to_gpu(res, 0, faiss.IndexIVFPQ(quantizer, d, nlist, m, 8, faiss.METRIC_INNER_PRODUCT))

# sample 5000 from db_vectors with shuffle
train_db_vectors = db_vectors.copy()
np.random.shuffle(train_db_vectors)
train_db_vectors = train_db_vectors[:500]

index.train(train_db_vectors)
index.add(db_vectors)

# 检索
D, I = index.search(query_vectors, k)  # 搜索最近的k个向量
# print(I[0])
# print(D[0])


WARNING clustering 500 points to 100 centroids: please provide at least 3900 training points
WARNING clustering 500 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 500 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 500 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 500 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 500 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 500 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 500 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 500 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 500 points to 256 centroids: please provide at least 9984 training points
WARNING clustering 500 points to 256 centroids: please provide at leas

In [20]:
k=120
trace1 = go.Scatter(x=np.arange(k), y=D[0][:], name='PQ')
trace2 = go.Scatter(x=np.arange(k), y=np.dot(query_vectors[0], db_vectors[I[0]].T), name='oriIP')
fig = go.Figure([trace1, trace2])
fig.show()


In [21]:
trace1 = go.Scatter(x=np.arange(k), y=D[1][:], name='PQ')
trace2 = go.Scatter(x=np.arange(k), y=np.dot(query_vectors[1], db_vectors[I[1]].T), name='oriIP')
fig = go.Figure([trace1, trace2])
fig.show()